In [ ]:
import pandas as pd
import pandas as pd
!pip install openai==0.28
import openai
from tqdm import tqdm
import time



from sklearn.metrics import accuracy_score, classification_report
x = pd.read_parquet('train.parquet', engine='pyarrow')


In [ ]:
x

,text,label
0,i am just so sick of feeling like this and i j...,3
1,i do know that i am not ready to feel any roma...,2
2,i would being feeling strange or nauseous so i...,4
3,i feel agitated of the time running so real slow,4
4,ive worked particularly hard with this year on...,4
...,...,...
15995,i need to make time for me and for doing thing...,2
15996,i feel this especially when i think back to si...,2
15997,i have discovered that every time i feel like ...,3
15998,i am starting to feel sorry for the muppet who...,0


In [ ]:
print(x["text"].unique())
print(x["label"].unique())

['i am just so sick of feeling like this and i just want opinions please nothing rude and imature'
 'i do know that i am not ready to feel any romantic love for anyone because i am not sure if i even believe in it anymore'
 'i would being feeling strange or nauseous so id think chewing gum would help'
 ...
 'i have discovered that every time i feel like a grumpy winter sloth if i go ride my bike i feel amazing'
 'i am starting to feel sorry for the muppet who has played patience on my train every morning this year'
 'i do get lucky i just feel nervous about how badly im going to get it next time']
[3 2 4 1 0 5]


In [ ]:
dataset = x[["text","label"]]
print(dataset.shape)
dataset.head(10)

(16000, 2)


,text,label
0,i am just so sick of feeling like this and i j...,3
1,i do know that i am not ready to feel any roma...,2
2,i would being feeling strange or nauseous so i...,4
3,i feel agitated of the time running so real slow,4
4,ive worked particularly hard with this year on...,4
5,i feel really welcomed into the flushing envir...,1
6,i read it outloud i don t feel alarmed at all ...,4
7,i feel a little bashful what is all this fuss ...,4
8,i am sure of where jesus my savior came from a...,4
9,i feel so loved encoding utf locale en ispriva...,2


In [ ]:
openai.api_key = "APIKEY"

In [ ]:
# Empty list to store selected DataFrames
selected_dfs = []
promptlist=[]
contextlist=[]

context_values = [2,4,5]

# Iterate over each context value
for context1 in context_values:
    # Filter DataFrame based on current context value
    context_df = dataset[dataset['label'] == context1]
    # Randomly select 10 records
    selected_df = context_df.sample(n=30, random_state=0)  # Set random_state for reproducibility
    # Append selected DataFrame to list
    selected_dfs.append(selected_df)

# Concatenate all selected DataFrames into one DataFrame
result_df = pd.concat(selected_dfs)

# Print result DataFrame
print(result_df)

                                                    text  label
11331  i feel so very blessed to have you all in my life      2
669    i love the extremes when my body aches for air...      2
11045  im such a bad person apology finally accepted ...      2
11785  i feel very passionate about many things my wi...      2
5818   i am pregnant why isn t it showing up i can st...      2
...                                                  ...    ...
2354   i feel like im saying i love the choreo of eve...      5
15671                   i feel this very berry funny luu      5
14154  i decided to give up the disgusting habit of s...      5
15406  i often feel surprised when reading a neville ...      5
12870  i am sitting here going over what i need to pa...      5

[90 rows x 2 columns]


In [ ]:
result_df.reset_index(drop=True, inplace=True)
print(result_df)

                                                 text  label
0   i feel so very blessed to have you all in my life      2
1   i love the extremes when my body aches for air...      2
2   im such a bad person apology finally accepted ...      2
3   i feel very passionate about many things my wi...      2
4   i am pregnant why isn t it showing up i can st...      2
..                                                ...    ...
85  i feel like im saying i love the choreo of eve...      5
86                   i feel this very berry funny luu      5
87  i decided to give up the disgusting habit of s...      5
88  i often feel surprised when reading a neville ...      5
89  i am sitting here going over what i need to pa...      5

[90 rows x 2 columns]


In [ ]:
prompt=[]
context=[]
GPTEmotion=[]


for i in range(0,result_df["text"].nunique()):
    message = result_df["text"][i]
    cont = result_df["label"][i]
    promptgpt = f"""Classify the following  message as the below output.
    Message: '{message}'.
    The output should only contain words from the following: 'sadness','joy','love','anger','fear','surprise'.
    """
    Context= """ Act as a Human Emotion Detector for a given message.
    I am going to provide you the message that can be defined as a specific emotion and you will respond with the output that contains only the one word emotion from 'sadness','joy','love','anger','fear','surprise'  """

    message = [ { 'role':'system' , 'content': Context},
              {'role':'user', 'content':promptgpt}]
    print("Before the sleep statement")
    time.sleep(20)
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    temperature = 0.0,
    messages=message
    )

    GPTRes = response.choices[0].message.content
    prompt.append(message)
    context.append(cont)
    GPTEmotion.append(GPTRes)




Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
Before the sleep statement
B

RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-VSwkfFiFtCr5sH2Su0tr77kN on requests per day (RPD): Limit 200, Used 200, Requested 1. Please try again in 7m12s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.

In [ ]:
d={'Prompt':prompt,'Context':context,'GPT Emotion':GPTEmotion}
final=pd.DataFrame(d)

In [ ]:
final['GPT Emotion'] = final['GPT Emotion'].str.lower()

In [ ]:
emotion_mapping = {'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

# Map emotions to numerical values
final['GPT_numerical_emotions'] = final['GPT Emotion'].map(emotion_mapping)

In [ ]:
final

,Prompt,Context,GPT Emotion,GPT_numerical_emotions
0,"[{'role': 'system', 'content': ' Act as a Huma...",2,love,2
1,"[{'role': 'system', 'content': ' Act as a Huma...",2,love,2
2,"[{'role': 'system', 'content': ' Act as a Huma...",2,anger,3
3,"[{'role': 'system', 'content': ' Act as a Huma...",2,love,2
4,"[{'role': 'system', 'content': ' Act as a Huma...",2,fear,4
5,"[{'role': 'system', 'content': ' Act as a Huma...",2,joy,1
6,"[{'role': 'system', 'content': ' Act as a Huma...",2,sadness,0
7,"[{'role': 'system', 'content': ' Act as a Huma...",2,joy,1
8,"[{'role': 'system', 'content': ' Act as a Huma...",2,fear,4
9,"[{'role': 'system', 'content': ' Act as a Huma...",2,surprise,5


In [ ]:
# Drop rows with missing values in the 'GPT_numerical_emotions' column
final.dropna(subset=['GPT_numerical_emotions'], inplace=True)

# Convert the 'GPT_numerical_emotions' column to integers
final['GPT_numerical_emotions'] = final['GPT_numerical_emotions'].astype(int)


In [ ]:
final

,Prompt,Context,GPT Emotion,GPT_numerical_emotions
0,"[{'role': 'system', 'content': ' Act as a Huma...",2,love,2
1,"[{'role': 'system', 'content': ' Act as a Huma...",2,love,2
2,"[{'role': 'system', 'content': ' Act as a Huma...",2,anger,3
3,"[{'role': 'system', 'content': ' Act as a Huma...",2,love,2
4,"[{'role': 'system', 'content': ' Act as a Huma...",2,fear,4
5,"[{'role': 'system', 'content': ' Act as a Huma...",2,joy,1
6,"[{'role': 'system', 'content': ' Act as a Huma...",2,sadness,0
7,"[{'role': 'system', 'content': ' Act as a Huma...",2,joy,1
8,"[{'role': 'system', 'content': ' Act as a Huma...",2,fear,4
9,"[{'role': 'system', 'content': ' Act as a Huma...",2,surprise,5


In [ ]:
# Compute accuracy
accuracy = accuracy_score(final['GPT_numerical_emotions'], final['Context'])

# Compute other classification metrics (e.g., precision, recall, F1-score)
classification_metrics = classification_report(final['GPT_numerical_emotions'], final['Context'])

# Print accuracy and classification report
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_metrics)

Accuracy: 0.375
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       0.00      0.00      0.00        11
           2       0.27      0.89      0.41         9
           3       0.00      0.00      0.00         6
           4       0.50      0.87      0.63        15
           5       0.00      0.00      0.00         3

    accuracy                           0.38        56
   macro avg       0.13      0.29      0.17        56
weighted avg       0.18      0.38      0.24        56



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
